In [1]:
import pickle
from configs import project_config
import numpy as np
import torch
import os
from tqdm import tqdm
from src.classes.ClassConditionalStbleTgtMarkovianPostMeanDiffTrainer import \
    ConditionalStbleTgtMarkovianPostMeanDiffTrainer
from src.generative_modelling.data_processing import train_and_save_recursive_diffusion_model
from src.generative_modelling.models.ClassVPSDEDiffusion import VPSDEDiffusion
from src.generative_modelling.models.TimeDependentScoreNetworks.ClassConditionalMarkovianTSPostMeanScoreMatching import \
    ConditionalMarkovianTSPostMeanScoreMatching
from utils.data_processing import init_experiment, cleanup_experiment
from utils.math_functions import generate_fQuadSin
from utils.resource_logger import ResourceLogger

NOTE: Redirects are currently not supported in Windows or MacOs.


In [2]:
# Data parameters
from configs.RecursiveVPSDE.Markovian_fBiPotDDims_NonSep.recursive_Markovian_PostMeanScore_fBiPot8DimsNS_T256_H05_tl_110data_StbleTgt import get_config
config = get_config()

In [4]:
assert (config.hurst == 0.5)
assert (config.early_stop_idx == 0)
assert (config.tdata_mult == 110)
assert (config.feat_thresh == 1./100.)
print(config.scoreNet_trained_path, config.dataSize)
rng = np.random.default_rng()
scoreModel = ConditionalMarkovianTSPostMeanScoreMatching(
    *config.model_parameters)
diffusion = VPSDEDiffusion(beta_max=config.beta_max, beta_min=config.beta_min)


/Users/marcos/GitHubRepos/FractionalBrownianMotion/src/generative_modelling/trained_models/trained_rec_ST_0010FTh_PM_MLP_2LFac_fBiPot_8DDimsNS_VPSDE_T256_Ndiff10000_Tdiff1000e+00_DiffEmbSz64_ResLay10_ResChan8_DiffHdnSz64_TrueHybd_TrueWghts_t00_dT3906e-03_025a_-20b_00c_MLP_H4_CUp20_tl110 40000


In [5]:
model_dir = "/".join(config.scoreNet_trained_path.split("/")[:-1]) + "/"
entered = False
for file in os.listdir(model_dir):
    if config.scoreNet_trained_path in os.path.join(model_dir, file) and "EE" in file:
        print(file)
        entered = True
        scoreModel.load_state_dict(torch.load(os.path.join(model_dir, file)))
assert entered

trained_rec_ST_0010FTh_PM_MLP_2LFac_fBiPot_8DDimsNS_VPSDE_T256_Ndiff10000_Tdiff1000e+00_DiffEmbSz64_ResLay10_ResChan8_DiffHdnSz64_TrueHybd_TrueWghts_t00_dT3906e-03_025a_-20b_00c_MLP_H4_CUp20_tl110_EENEp260


In [ ]:
def true_drift(prev, num_paths, config):
    assert (prev.shape == (num_paths, config.ndims))
    drift_X = -(4. * np.array(config.quartic_coeff) * np.power(prev,
                                                                 3) + 2. * np.array(config.quad_coeff) * prev + np.array(config.const))
    xstar = np.sqrt(
        np.maximum(1e-12, -np.array(config.quad_coeff) / (2.0 * np.array(config.quartic_coeff))))
    s2 = (config.scale * xstar) ** 2 + 1e-12  # (D,) or (K,1,D)
    diff = prev ** 2 - xstar ** 2  # same shape as prev
    phi = np.exp(-(diff ** 2) / (2.0 * s2 * xstar ** 2 + 1e-12))
    phi_prime = phi * (-2.0 * prev * diff / ((config.scale ** 2) * (xstar ** 4 + 1e-12)))
    nbr = np.roll(phi, 1, axis=-1) + np.roll(phi, -1, axis=-1)  # same shape as phi
    drift_X = drift_X - 0.5 * config.coupling * phi_prime * nbr
    return drift_X[:, np.newaxis, :]

In [ ]:
def experiment_multivar_score_based_MLP_drift_OOS(score_model, num_diff_times, diffusion, num_paths, prev,
                                       ts_step, config,
                                       device, onlyGauss):
    """ Computes drift using MLP score network when features obtained from LSTM directly """

    score_model = score_model.to(device)
    num_taus = 100
    Ndiff_discretisation = config.max_diff_steps
    assert (prev.shape == (num_paths, config.ndims))
    assert (prev[0, :].shape[0] == config.ts_dims)
    features_tensor = torch.stack([torch.tensor(prev, dtype=torch.float32) for _ in range(1)], dim=0).reshape(
        num_paths * 1, 1, -1).to(device)
    assert (features_tensor.shape[0] == num_paths)
    vec_Z_taus = diffusion.prior_sampling(shape=(num_paths * num_taus, 1, config.ts_dims)).to(device)

    diffusion_times = torch.linspace(config.sample_eps, 1., config.max_diff_steps)
    difftime_idx = Ndiff_discretisation - 1
    for difftime_idx in tqdm(np.arange(num_diff_times - 1, num_diff_times - es - 1, -1)): #difftime_idx >= num_diff_times - es:
        d = diffusion_times[difftime_idx].to(device)
        diff_times = torch.stack([d for _ in range(num_paths)]).reshape(num_paths * 1).to(device)
        eff_times = diffusion.get_eff_times(diff_times=diff_times).unsqueeze(-1).unsqueeze(-1).to(device)
        vec_diff_times = torch.stack([diff_times for _ in range(num_taus)], dim=0).reshape(num_taus * num_paths)
        vec_eff_times = torch.stack([eff_times for _ in range(num_taus)], dim=0).reshape(num_taus * num_paths, 1, 1)
        vec_conditioner = torch.stack([features_tensor for _ in range(num_taus)], dim=0).reshape(
            num_taus * num_paths,
            1, -1)
        score_model.eval()
        with torch.no_grad():
            if onlyGauss:
                scoreEval_vec_Z_taus = diffusion.prior_sampling(shape=vec_Z_taus.shape).to(device)
            else:
                scoreEval_vec_Z_taus = vec_Z_taus
            vec_predicted_score = score_model.forward(times=vec_diff_times, eff_times=vec_eff_times,
                                                      conditioner=vec_conditioner, inputs=scoreEval_vec_Z_taus)
        vec_scores, vec_drift, vec_diffParam = diffusion.get_conditional_reverse_diffusion(x=vec_Z_taus,
                                                                                           predicted_score=vec_predicted_score,
                                                                                           diff_index=torch.Tensor(
                                                                                               [int((
                                                                                                       num_diff_times - 1 - difftime_idx))]).to(
                                                                                               device),
                                                                                           max_diff_steps=Ndiff_discretisation)
        # assert np.allclose((scores- predicted_score).detach(), 0)
        beta_taus = torch.exp(-0.5 * eff_times[0, 0, 0]).to(device)
        sigma_taus = torch.pow(1. - torch.pow(beta_taus, 2), 0.5).to(device)
        final_mu_hats = (vec_Z_taus / (ts_step * beta_taus)) + ((
                                                                        (torch.pow(sigma_taus, 2) + (
                                                                                torch.pow(beta_taus * config.diffusion,
                                                                                          2) * ts_step)) / (
                                                                                ts_step * beta_taus)) * vec_scores)

        assert (final_mu_hats.shape == (num_taus * num_paths, 1, config.ts_dims))
        means = final_mu_hats.reshape((num_taus, num_paths, config.ts_dims))
        assert (means.shape == (num_taus, num_paths, config.ts_dims))
        # print(vec_Z_taus.shape, vec_scores.shape)
        vec_z = torch.randn_like(vec_drift).to(device)
        vec_Z_taus = vec_drift + vec_diffParam * vec_z
    return means.mean(dim=0).reshape(num_paths, 1, num_diff_times, config.ts_dims).cpu().numpy()


In [ ]:
def weak_error_computation(config, score_network, num_time_steps, onlyGauss):
    num_diff_times = num_time_steps
    rmse_quantile_nums = 2
    num_paths = 100
    num_time_steps = 256
    all_true_states = np.zeros(shape=(rmse_quantile_nums, num_paths, 1 + num_time_steps, num_diff_times, config.ndims))
    all_global_states = np.zeros(shape=(rmse_quantile_nums, num_paths, 1 + num_time_steps, num_diff_times, config.ndims))
    all_local_states = np.zeros(shape=(rmse_quantile_nums, num_paths, 1 + num_time_steps, num_diff_times, config.ndims))
    deltaT = config.deltaT
    for quant_idx in tqdm(range(rmse_quantile_nums)):
        score_network.eval()
        initial_state = np.repeat(np.atleast_2d(config.initState)[np.newaxis, :], num_paths, axis=0)
        assert (initial_state.shape == (num_paths, 1, num_diff_times, config.ndims))

        true_states = np.zeros(shape=(num_paths, 1 + num_time_steps, num_diff_times, config.ndims))
        global_states = np.zeros(shape=(num_paths, 1 + num_time_steps, num_diff_times, config.ndims))
        local_states = np.zeros(shape=(num_paths, 1 + num_time_steps, num_diff_times, config.ndims))

        # Initialise the "true paths"
        true_states[:, [0], :] = initial_state + 0.00001 * np.random.randn(*initial_state.shape)
        # Initialise the "global score-based drift paths"
        global_states[:, [0], :] = true_states[:, [0], :]
        local_states[:, [0], :] = true_states[:, [0], :]  # np.repeat(initial_state[np.newaxis, :], num_diff_times, axis=0)

        # Euler-Maruyama Scheme for Tracking Errors
        for i in range(1, num_time_steps + 1):
            eps = np.random.randn(num_paths, 1, num_diff_times, config.ndims) * np.sqrt(deltaT)*config.diffusion
            assert (eps.shape == (num_paths, 1, num_diff_times, config.ndims))
            true_mean = true_drift(true_states[:, i - 1, :, :], num_paths=num_paths, config=config)
            denom = 1.
            local_mean = experiment_multivar_score_based_MLP_drift_OOS(
                score_model=score_network,
               num_diff_times=num_diff_times,
                diffusion=diffusion,
                num_paths=num_paths, ts_step=deltaT,
                config=config,
                device=score_network.device,
                prev=true_states[:, i - 1,:, :], onlyGauss=onlyGauss)

            global_mean = experiment_multivar_score_based_MLP_drift_OOS(score_model=score_network,
                                                                                  num_diff_times=num_diff_times,
                                                                                  diffusion=diffusion,
                                                                                  num_paths=num_paths,
                                                                                  ts_step=deltaT, config=config,
                                                                                  device=score_network.device,
                                                                                  prev=global_states[:, i - 1,:,  :], onlyGauss=onlyGauss)

            true_states[:, [i], :,:] = (true_states[:, [i - 1], :, :] \
                                     + true_mean * deltaT \
                                     + eps)/denom
            local_states[:, [i], :,:] = (true_states[:, [i - 1], :, :] + local_mean * deltaT + eps)/denom
            global_states[:, [i], :,:] = (global_states[:, [i - 1], :, :] + global_mean * deltaT + eps)/denom

        all_true_states[quant_idx, :, :, :, :] = true_states
        all_local_states[quant_idx, :, :, :,:] = local_states
        all_global_states[quant_idx, :, :,:, :] = global_states
    return all_true_states, all_local_states, all_global_states

In [22]:
onlyGauss = True
true_state, local_states, global_states = weak_error_computation(score_network=scoreModel, num_time_steps=config.max_diff_steps, onlyGauss=onlyGauss ,config=config)

Beta Min :  0.0
Using CPU

/Users/marcos/GitHubRepos/FractionalBrownianMotion/src/generative_modelling/trained_models/trained_rec_ST_0002FTh_PM_MLP_2LFac_NFMReg_fQuadSinHF_VPSDE_T256_Ndiff10000_Tdiff1000e+00_DiffEmbSz64_ResLay10_ResChan8_DiffHdnSz64_TrueHybd_TrueWghts_t00_dT3906e-03_05a_004b_250c_MLP_H4_CUp20_tl110


 69%|██████▊   | 6868/10000 [25:20<11:33,  4.52it/s]  

KeyboardInterrupt



In [5]:
remote_file_path = project_config.ROOT_DIR+"data/" #/Users/marcos/Library/CloudStorage/OneDrive-ImperialCollegeLondon/StatML_CDT/Year2/DiffusionModels/ExperimentResults/"
remote_file_path


'/Users/marcos/GitHubRepos/FractionalBrownianMotion/data/'

In [ ]:
np.save(remote_file_path + "8DDimsBiPotNonSep_optimal_tau_experiment_DGauss.npy", true_state)

In [ ]:
DGauss = np.load(remote_file_path + "QuadSinHF_optimal_tau_experiment_DGauss.npy", allow_pickle=True)
DNonGauss = np.load(remote_file_path + "QuadSinHF_optimal_tau_experiment_DNonGauss.npy", allow_pickle=True)
Xshape = 256
Xs = np.linspace(-1.5, 1.5, num=Xshape)
true_drifts = (-2. * config.quad_coeff * Xs + config.sin_coeff * config.sin_space_scale * np.sin(config.sin_space_scale * Xs))[:, np.newaxis, np.newaxis, np.newaxis]
Gauss_errors = np.mean(DGauss, axis=2) - true_drifts
assert np.all(Gauss_errors.shape == (Xshape, DGauss.shape[1], config.ts_dims))
NonGauss_errors = np.mean(DNonGauss, axis=2) - true_drifts
assert np.all(NonGauss_errors.shape == (Xshape, DNonGauss.shape[1], config.ts_dims))
Gauss_errors = np.sum(np.power(Gauss_errors,2), axis=-1)
NonGauss_errors = np.sum(np.power(NonGauss_errors,2), axis=-1)